In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, Trainer
from peft import PeftModel, PeftConfig
from huggingface_hub import login
import torch

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [4]:
def load_model_tokenizer(model_id, torch_dtype):

    torch_dtype = eval(torch_dtype)
    tokenizer = AutoTokenizer.from_pretrained(model_id, device_map=device)
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map=device, torch_dtype=torch_dtype)
    tokenizer.pad_token = tokenizer.eos_token

    model.resize_token_embeddings(len(tokenizer))
    return model, tokenizer


def merge_model(model, pathPeft): 

    new_model = PeftModel.from_pretrained(model, pathPeft).to(device)

    merged_model = new_model.merge_and_unload()
    print("MERGED")
    return merged_model

In [5]:
peft_path = "/cluster/home/terjenf/norwAI_All/results/final/mistral_7B_exp_1_1722629952"
model_id = "NorLLM-AI/NorMistral-7B"

In [6]:
model_id = "NorwAI/NorwAI-Mistral-7B"

In [7]:
model, tokenizer = load_model_tokenizer(model_id, "torch.float32")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
model.device

device(type='cpu')

In [7]:
merged_model = merge_model(model, peft_path)

MERGED


In [8]:
merged_path = "/cluster/home/terjenf/norwAI_All/llm_training/merged_models/merged_mistral_7B_exp_1_1722629952"
tokenizer

LlamaTokenizerFast(name_or_path='NorwAI/NorwAI-Mistral-7B', vocab_size=67993, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [10]:
tokenizer.save_vocabulary(merged_path)

('/cluster/home/terjenf/norwAI_All/llm_training/merged_models/merged_mistral_7B_exp_1_1722629952/tokenizer.model',)

In [9]:


#merged_model.save_pretrained(merged_path)
tokenizer.save_pretrained(merged_path)

TypeError: Object of type device is not JSON serializable

In [27]:
def push_model_tokenizer(model_name, commit_message, merged_model, tokenizer):
    merged_model.push_to_hub(model_name, commit_message=commit_message)
    tokenizer.push_to_hub(model_name, commit_message=commit_message)
    print("DONE")

In [26]:
model_name = 'finetuned-lora-NorMistral7B-Summary'
commit_message = 'merged lora and pre trained'

In [28]:
push_model_tokenizer(model_name, commit_message, merged_model, tokenizer)

[2024-08-27 10:30:16,386] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/cluster/home/terjenf/.conda/envs/vgdebatt/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-devel package with yum
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


model-00002-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/4.77G [00:00<?, ?B/s]

...

model-00005-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

TypeError: Object of type device is not JSON serializable